# Doha exceedance walkthrough

This notebook mirrors the paper's workflow for a single site (Doha International Airport) using live station data. Code cells carry inline comments for SMEs who want to trace each step.

In [ ]:
from datetime import date
import pandas as pd
from climate_analysis.config import load_thresholds
from climate_analysis.sites import get_site
from climate_analysis.data_access import StationClient
from climate_analysis.processing import compute_tx01_from_daily
from climate_analysis.analysis import return_time_bootstrap

# Load site metadata and default thresholds from config
site = get_site("QAT")
default_thresholds = load_thresholds()["default_thresholds_c"]

# Users can add more thresholds here; duplicates are automatically deduped
user_thresholds = [48.0]
thresholds = sorted({*default_thresholds, *user_thresholds})

print(f"Using thresholds: {thresholds}")

In [ ]:
# Download daily Tmax using Meteostat (no API keys required).
# Dates cover the paper's observed period for Doha.
station_client = StationClient()
daily_tmax = station_client.fetch_daily_tmax(
    site=site,
    start=date(1983, 1, 1),
    end=date(2023, 12, 31),
)
daily_tmax.head()

In [ ]:
# Compute tx01: the warmest day of each year.
tx01 = compute_tx01_from_daily(daily_tmax)
tx01.describe()

In [ ]:
# Fit GEV and compute exceedance probabilities/return times for each threshold.
rows = []
for threshold in thresholds:
    stats_row = return_time_bootstrap(tx01, threshold_c=threshold, n_boot=300)
    rows.append({"threshold_c": threshold, **stats_row})
pd.DataFrame(rows)

In [ ]:
# Quick plot: annual maxima with thresholds overlaid for visual intuition.
ax = tx01.plot(kind="line", figsize=(10, 4), title="Annual maximum daily temperature (Doha)")
for thr in thresholds:
    ax.axhline(thr, color="red" if thr >= 50 else "orange", linestyle="--", alpha=0.6, label=f"{thr} °C")
ax.legend()
ax.set_ylabel("Tmax (°C)")
ax.figure.tight_layout()
ax.figure